In [12]:
from enum import Enum, auto
from dataclasses import fields
import ipywidgets as wgt
from IPython import display
from pathlib import Path
import json
from typing import Any
from datetime import datetime

import numpy as np

In [1]:
from libs.environment.cost_calculators import *
from libs.optimizers.algorithms.genetic.operators.mutations import *
from libs.optimizers.algorithms.genetic.operators.crossovers import *
from libs.optimizers.algorithms.genetic.operators.fixers import *
from libs.optimizers.algorithms.genetic.population import Population
from libs.optimizers.algorithms.genetic.population.generators import *
from libs.optimizers.algorithms.genetic.population.parent_selectors import *
from libs.optimizers.algorithms.genetic.population.population_selectors import *
from libs.optimizers.algorithms.genetic.operators.fixers import *
from libs.solution.initial_solution_creators.heuristic import *
from libs.solution.initial_solution_creators.random import *

## TSP

In [3]:
RND_SEED = 0
rng = np.random.default_rng(seed=RND_SEED)

In [9]:
class CreatorType(Enum):
    HEURISTIC = auto()
    RANDOM = auto()


def generate_initial_population(
    size: int,
    cost_mx: np.ndarray,
    prob_of_swap: float,
    creator_t: CreatorType = CreatorType.RANDOM,
    initial_vx: int = 0,
):
    """
    If `creator_t` is `HEURISTIC`, generates one solution and its mutated copies.
    If `creator_t` is `RANDOM`, generates `n` random solutions.
    """
    if creator_t == CreatorType.HEURISTIC:
        initial_solution = create_tsp_solution_nearest_neighbour(cost_mx, initial_vx)
        mutated_solutions = (
            mutate_swap(initial_solution, p=prob_of_swap, rng=rng)[0]
            for _ in range(size - 1)
        )
        return [initial_solution, *mutated_solutions]

    return [create_tsp_solution_random(cost_mx, initial_vx, rng) for _ in range(size)]

In [ ]:
ENV_LOAD_DIR = Path("./data/environments")
loaded_data: list[dict[str, Any]] = [{}]
file_to_load_name_wgt = wgt.Text(
    description="file to load", placeholder="type json filename"
)
load_data_btn = wgt.Button(description="load data")


def load_data_cb(_):
    load_path = ENV_LOAD_DIR / file_to_load_name_wgt.value
    if not load_path.exists():
        raise FileNotFoundError(str(load_path))
    with load_path.open("r") as f:
        loaded_data[0] = json.load(f)

load_data_btn.on_click(load_data_cb)
display(file_to_load_name_wgt)
display(load_data_btn)

In [ ]:
SAVE_DIR = Path("./data/populations/tsp")
SAVE_PATH_TIME_FMT = "%Y-%m-%d_%H-%M-%S"

cost_mx = loaded_data[0]["travel_time"]
initial_vx_wgt = wgt.BoundedIntText(description="initial vertex", value=0, min=0, max=cost_mx.shape[0]-1)
pop_size_wgt = wgt.BoundedIntText(description="population size", min=0, max=100)
prob_of_swap_wgt = wgt.BoundedFloatText(description="swap prob", value=0.1, min=0, max=1)
creator_t_wgt = wgt.Dropdown(description="creator", value=CreatorType.RANDOM, options=[(e.name, e) for e in CreatorType])
generate_btn = wgt.Button(description="generate population")

generated_population = [[]]
initial_costs = [[]]

def create_population_cb(_):
    """
    Generates population and saves it.
    """
    initial_vx = initial_vx_wgt.value
    new_pop = generate_initial_population(
        size=pop_size_wgt.value,
        cost_mx=cost_mx,
        creator_t=creator_t_wgt.value,
        prob_of_swap=prob_of_swap_wgt.value,
        initial_vx=initial_vx,
    )
    fixed_pop_and_fix_results = [fix_tsp(c, cost_mx, initial_vx) for c in new_pop]
    fixed_pop = [x[0] for x in fixed_pop_and_fix_results]
    fix_results = [x[1] for x in fixed_pop_and_fix_results]
    fix_results_parsed = [{"no_of_errors": fr.no_of_errors, "fix_status": fr.fix_status.name} for fr in fix_results]
    fixed_pop_costs = [calculate_total_cost(cost_tsp_gen(s, cost_mx, initial_vx)) for s in fixed_pop]
    generated_population[0] = fixed_pop
    initial_costs[0] = fixed_pop_costs
    now_str = datetime.now().strftime(SAVE_PATH_TIME_FMT)
    save_path = SAVE_DIR / f"population_tsp_{now_str}.json"
    with save_path.open("w") as f:
        json.dump({"population": fixed_pop, "costs": fixed_pop_costs, "fix_results": fix_results_parsed}, f)


tsp_gen_widgets = wgt.VBox(children=[
    initial_vx_wgt, pop_size_wgt, prob_of_swap_wgt, creator_t_wgt
])

generate_btn.on_click(create_population_cb)

display(tsp_gen_widgets)
display(generate_btn)

## TODOs

- MTSP - jak wyżej
- MVMTSP - jak wyżej, inne mutatory, liczyć koszt na podst. wizyt w wierzchołkach
- IRP - inne mutatory, koszt na podst. zostawianego towaru, który odnawia się
  po wizycie w depocie (optymalizacja - powrót do depota najkrótszą drogą (Dijkstra)
  po zużyciu towaru)
- jupyterowy kreator configów
- scheduler dla eksperymentów